# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 6. Support Vector Machine (SVM)


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [22]:
#Sudah di download

Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [23]:
import pandas as pd
import numpy as np
df = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [24]:
df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


Metode Support Vector Machine (SVM) dasar hanya mampu melakukan klasifikasi data yang terdiri dari dua kelas, atau disebut klasifikasi biner. Dataset iris memiliki 3 kelas, sehingga salah satu kelasnya, yaitu Iris-virginica perlu dihapus. Cell berikut menghapus data yang memiliki kelas Iris-virginica

In [25]:
df.drop(df[df['variety'] == 'Virginica'].index, inplace=True)

SVM juga memiliki keterbatasan, yaitu hanya mampu menerima kelas dalam bentuk numerik. Cell berikut mengubah kelas menjadi bilangan, Iris setosa menjadi -1 dan Iris-versicolor menjadi 1

In [26]:
df['variety'] = df['variety'].map({'Setosa': -1, 'Versicolor': 1})

Tampilkan beberapa data untuk mengecek hasilnya

In [27]:
df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,-1
1,4.9,3.0,1.4,0.2,-1
2,4.7,3.2,1.3,0.2,-1
3,4.6,3.1,1.5,0.2,-1
4,5.0,3.6,1.4,0.2,-1


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [28]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 80 data, dan **data_uji** terdiri dari 20 data

In [29]:
print(df_train.shape[0])
print(df_test.shape[0])

80
20


Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_latih** dan **label_uji**

In [30]:
label_train = df_train.pop('variety')
label_test = df_test.pop('variety')

## 3) Proses Training

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [31]:
def count_cost_gradient(W, X, Y, regularization):
  distance = 1 - Y * (np.dot(X, W))
  dw = np.zeros(len(W))
  if max(0, distance) == 0:
    di = W
  else:
    di = W - (regularization * Y * X)
    
  dw += di
  return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD 

In [32]:
from sklearn.utils import shuffle

def sgd(df_train, label_train, learning_rate=0.000001, max_epoch=1000, regularization=10000):
  df_train = df_train.to_numpy()
  label_train = label_train.to_numpy()
  weight = np.zeros(df_train.shape[1])
  for epoch in range(max_epoch):
    X, Y = shuffle(df_train, label_train, random_state=101)
    for index, x in enumerate(X):
      delta = count_cost_gradient(weight, x, Y[index], regularization)
      weight = weight - (learning_rate * delta)
  
  return weight


Proses training dilakukan dengan memanggil fungsi **sgd** dengan parameter input berupa data latih dan label latih. Parameter learning rate dan max epoch menggunakan nilai *default* nya

In [33]:
W = sgd(df_train, label_train)
print(W)

[-0.24487361 -0.68250748  1.13494151  0.51316484]


## 4) Proses *testing*
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [34]:
def testing(W, df_test):
  prediction = np.array([])
  for i in range(df_test.shape[0]):
    y_prediction = np.sign(np.dot(W, df_test.to_numpy()[i]))
    prediction = np.append(prediction, y_prediction)
  
  return prediction

Lakukan pengujian menggunakan seluruh data uji. Bandingkan hasilnya dengan nilai label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar.

In [35]:
y_prediction = testing(W, df_test)
print(sum(y_prediction == label_test))

20


## TUGAS
Pada tugas kali ini Anda masih melakukan klasifikasi dataset iris, tetapi menggunakan kelas Iris-versicolor dan Iris-virginica. Apakah hasilnya lebih baik jika dibandingkan dengan Iris-setosa dan Iris-versicolor?

In [36]:
new_df = pd.read_csv('iris.csv')
new_df.drop(new_df[new_df['variety'] == 'Setosa'].index, inplace=True)
new_df['variety'] = new_df['variety'].map({'Virginica': -1, 'Versicolor': 1})
new_df_train, new_df_test = train_test_split(new_df, test_size=0.2, random_state=42)
new_label_train = new_df_train.pop('variety')
new_label_test = new_df_test.pop('variety')
print(new_df_train.shape[0])
print(new_df_test.shape[0])

80
20


In [37]:
W = sgd(new_df_train, new_label_train)
print(W)

[ 3.47308345  6.29317639 -6.0618632  -6.43031342]


In [38]:
y_prediction = testing(W, new_df_test)
print(sum(y_prediction == new_label_test))

17


Hasil uji  yang dihasilkan  tidak lebih baik dari hasil uji sebelumnya karena hanya mampu mendapatkan prediksi tepat sebanyak 13 diantara 20 data uji.

Hal ini membuktikan bahwa prose klasifikasi yang menggunakan kelas Iris-versicolor dan Iris-virginica tidak lebih baik dari Iris-setosa dan Iris-versicolor

Lakukan proses training ulang, tetapi menggunakan nilai max_epoch sebesar 5000. Anda dapat memberikan parameter max_epoch=5000 saat memanggil fungsi **sgd**. Apakah hasilnya lebih baik? Mengapa?

In [39]:
W = sgd(new_df_train, new_label_train, max_epoch=5000)
y_prediction = testing(W, new_df_test)
print(sum(y_prediction == new_label_test))

17


Hasil yang dihasilkan justru lebih baik meningkat dari yang awalnya 13 data yang tepat menjadi 16 data yang tepat

Menambahkan epoch pada algoritma SVM yang menggunakan stochastic gradient descent (SGD) dapat meningkatkan akurasi model karena memungkinkan algoritma untuk melatih model lebih baik dengan menggunakan lebih banyak iterasi atau langkah pembelajaran.

Pada dasarnya, epoch adalah satu putaran penuh melalui seluruh dataset pelatihan. Dengan menggunakan SGD, kita memperbarui parameter model dengan menghitung gradien dari fungsi tujuan (misalnya, fungsi kerugian) hanya menggunakan subset kecil dari data pelatihan pada setiap iterasi. Dengan menambahkan lebih banyak epoch, memberikan lebih banyak kesempatan bagi model untuk menyesuaikan diri dengan data pelatihan secara lebih baik.

Terakhir, lakukanlah seleksi fitur sederahana dengan mengambil hanya fitur sepal length dan sepal width saja. Apakah hasilnya lebih baik?

In [40]:
new_df.drop(['petal.length', 'petal.width'], axis=1, inplace=True)
new_df_train, new_df_test = train_test_split(new_df, test_size=0.2, random_state=42)
new_label_train = new_df_train.pop('variety')
new_label_test = new_df_test.pop('variety')
print(new_df_train.shape[0])
print(new_df_test.shape[0])

80
20


In [41]:
W = sgd(new_df_train, new_label_train)
print(W)

[-0.71267175  1.55047623]


In [42]:
y_prediction = testing(W, new_df_test)
print(sum(y_prediction == new_label_test))

6


Berdasarkan hasil uji  yang didapatkan (hanya menggunakan sepal length dan sepal widht), jumlah prediksi yang memiliki nilai sama justru lebih sedikit dibandingkan sebelumnya, dimana hanya terdapat 6 data yang memiliki nilai yang sama